In [1]:
import numpy as np 
import math
from decimal import *
getcontext().prec = 10

matriz_posicao = [[0.280, 0, 0.300]]
matriz_forca_max  = [[0, 16000, 12000]]
matriz_forca_min = [[0, 0, 0]]
matriz_momento = ''
d= ''
tensao_ruptura = 1000e6
lim_esc = Decimal(800e6)
aco = 'laminado_quente'
carga = 'axial'
conf = 50
q = 1
kt = 1
n = Decimal(2)
temp = 80
tensao_adm = lim_esc/n

# 1 - Sorderberg
# 2 - Goodman
# 3 - Gerber
# 4 - ASME
# 5 - Escoamento Ampliado
criterio = 1

In [2]:
def prod_vet(matriz_posicao, matriz_forca, matriz_momento):
  i = 0
  a = []

  while i < len(matriz_posicao):
      prod = np.cross(matriz_posicao[i], matriz_forca[i])
      prod = a.append(prod)
      prod = sum(a)
      i = i + 1

  if matriz_momento == '':
    matriz_forca = np.sum(matriz_forca, axis=0)
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    f = matriz_forca[2]
  else:
    matriz_forca = np.sum(matriz_forca, axis=0)
    matriz_momento = np.sum(matriz_momento, axis=0)
    prod = prod + matriz_momento
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    f = matriz_forca[2]
  
  print(f'F = {matriz_forca[0]:.2f} i + {matriz_forca[1]:.2f} j + {matriz_forca[2]:.2f} k')
  print(f'M = {prod[0]:.2f} i + {prod[1]:.2f} j + {prod[2]:.2f} k')
    

 
  return m,t,f

In [3]:
#momento máximo e torque máximo
x = prod_vet(matriz_posicao, matriz_forca_max, matriz_momento)

f_max = (x[2])
t_max = Decimal(x[1])
m_max = Decimal(x[0])


#momento mínimo e torque mínimo
y = prod_vet(matriz_posicao, matriz_forca_min, matriz_momento)

f_min = (y[2])
t_min = Decimal(y[1])
m_min = Decimal(y[0])

F = 0.00 i + 16000.00 j + 12000.00 k
M = -4800.00 i + -3360.00 j + 4480.00 k
F = 0.00 i + 0.00 j + 0.00 k
M = 0.00 i + 0.00 j + 0.00 k


In [4]:
# determinar ka
def ka(aco):
  if aco == 'forjado':
    a = 272
    b = -0.995
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'laminado_quente':
    a = 57.7
    b = -0.718
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'usinado_ou_laminado_frio':
    a = 4.51
    b = -0.265
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'retificado':
    a = 1.58
    b = -0.085
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'espelhado':
    a = 1
    b = 0
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  return ka

if ka(aco) > 1:
  ka = 1
else:
  ka = ka(aco)

In [5]:
# determinar kc
def kc(conf):
  if conf == 50:
    kc = 1
    
  elif conf == 90:
      kc = 0.897

  elif conf == 95:
      kc = 0.868

  elif conf == 98:
      kc = 0.836

  elif conf == 99:
      kc = 0.814

  elif conf == 99.99:
      kc = 0.702

  elif conf == 99.9999:
      kc = 0.620

  elif conf == 99.99999:
      kc = 0.584

  return kc

kc = kc(conf)

In [6]:
# determinar kd
def kd(temp):
  kd = 0.9877 + (0.6507 * (10**-3) * temp) - (0.3414 * (10**-5) * (temp**2)) + (0.5621 * (10**-8) * (temp**3)) - (6.246 * (10**-12) * (temp**4))

  return kd

if kd(temp) > 1:
  kd = 1
else:
  kd = kd(temp)

In [7]:
# determinar kf
kf = 1

In [8]:
tensao_cisalhante_esc = Decimal(0.577) * lim_esc

In [9]:
# determinar o kp
def kp(q,kt):
    kp = 1 + (q * (kt - 1))
    return kp  

In [10]:
kp = Decimal(kp(q,kt))

In [11]:
pi = Decimal(math.pi)

In [12]:
def tensao_normal_max(d):
  return (kp * ((Decimal(32) * m_max) / (Decimal(pi) * (d**Decimal(3))))) + (kp * ((Decimal(4) * f_max) / (Decimal(pi) * (d**Decimal(2)))))

def tensao_normal_min(d):
  return (kp * ((Decimal(32) * m_min) / (Decimal(pi) * (d**Decimal(3))))) + (kp * ((Decimal(4) * f_min) / (Decimal(pi) * (d**Decimal(2)))))

def tensao_cisalhante_max(d):
  return (kp * ((Decimal(16) * t_max) / (Decimal(pi) * (d**Decimal(3)))))

def tensao_cisalhante_min(d):
  return (kp * ((Decimal(16) * t_min) / (Decimal(pi) * (d**Decimal(3)))))

def tensao_normal_var(d):
  return (tensao_normal_max(d) - tensao_normal_min(d)) / Decimal(2)

def tensao_normal_media(d):
  return (tensao_normal_max(d) + tensao_normal_min(d)) / Decimal(2)

def tensao_cisalhante_media(d):
  return (tensao_cisalhante_max(d) + tensao_cisalhante_min(d)) / Decimal(2)

def tensao_cisalhante_var(d):
  return (tensao_cisalhante_max(d) - tensao_cisalhante_min(d)) / Decimal(2)

#critério de sorderberg
if criterio == 1:
  def tensao_normal(d):
    return tensao_normal_media(d) + (tensao_normal_var(d) * (lim_esc/lim_normal_res))

  def tensao_cisalhante(d):
    return tensao_cisalhante_media(d) + (tensao_cisalhante_var(d) * (tensao_cisalhante_esc/lim_cisalhante_res))

#critério modificado de goodman
elif criterio == 2:
  def tensao_normal(d):
    return lim_esc * ((tensao_normal_var(d) / lim_normal_res) + (tensao_normal_media(d) / Decimal(tensao_ruptura)))
  
  def tensao_cisalhante(d):
    return tensao_cisalhante_esc * ((tensao_cisalhante_var(d) / lim_cisalhante_res) + (tensao_cisalhante_media(d) / (Decimal(tensao_ruptura)*Decimal(0.8))))

#critério gerber
elif criterio == 3:
  def tensao_normal(d):
    a1 = Decimal(1)
    b1 = (lim_esc * tensao_normal_var(d)) / lim_normal_res
    c1 = ((lim_esc * tensao_normal_media(d)) / Decimal(tensao_ruptura))**Decimal(2)
    delta1 = b1*b1 - (Decimal(4)*a1*(-c1))
    return (Decimal(-b1) - Decimal(math.sqrt(delta1)) ) / (Decimal(2)*a1)

  def tensao_cisalhante(d):
    a2 = Decimal(1)
    b2 = (tensao_cisalhante_esc * tensao_cisalhante_var(d)) / lim_cisalhante_res
    c2 = ((tensao_cisalhante_esc * tensao_cisalhante_media(d)) / (Decimal(tensao_ruptura) * Decimal(0.8)))**Decimal(2) 
    delta2 = b2*b2 - (4*a2*(-c2))
    return (Decimal(-b2) - Decimal(math.sqrt(delta2)) ) / (Decimal(2)*a2)
    
    

#critério ASME
elif criterio == 4:
  def tensao_normal(d):
    return math.sqrt((((lim_esc * tensao_normal_var(d))/lim_normal_res)**2) + ((tensao_normal_media(d))**2))
  
  def tensao_cisalhante(d):
    return math.sqrt((((tensao_cisalhante_esc * tensao_cisalhante_var(d))/lim_cisalhante_res)**2) + ((tensao_cisalhante_media(d))**2))

#critério escoamento ampliado
elif criterio == 5:
  def tensao_normal(d):
    return tensao_normal_var(d) + tensao_normal_media(d)

  def tensao_cisalhante(d):
    return tensao_cisalhante_var(d) + tensao_cisalhante_media(d)
  
  

In [13]:
d0 = ((Decimal(32)/(pi*Decimal(tensao_adm)))*(Decimal(math.sqrt((m_max**Decimal(2))+((Decimal(3)/Decimal(4))*(t_max**Decimal(2)))))))**(Decimal(1)/Decimal(3))


In [14]:
def f(d):
  return Decimal(math.sqrt(((Decimal(tensao_normal(d)))**(Decimal(2))) + ((Decimal(3))* ((Decimal(tensao_cisalhante(d)))**(Decimal(2)))))) - (Decimal(lim_esc)/Decimal(n))

In [15]:
#encontrar o valor da derivada
def flin(f, d):  
  if d == 0:
    d = 1
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  else:
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  return flin

In [16]:
def newton(f, d0, epsilon, maxIter=20):
    if abs(f(d0)) <= epsilon:
      return (d0)

    print("k\t d0\t\t f(d0)") #cabeçalho da tabela que retornará com os valores embaixo. \t é o espaçamento que dou para ficar visual.
    
    k=1
    while k <= maxIter:
      d1 = d0 - f(d0)/flin(f,d0)
        
      print("%d\t%e\t%e" % (k, d1,f(d1)))

      if abs(f(d1)) <= epsilon:
        return d1      
      
      d0 = d1
      k = k + 1
    
    print('ERRO: Número máximo de iterações atingido')
    
    return d1  

In [17]:
parar = False

while parar == False:  
  if 2.79 <=  d0*10**3 <= 51:
    kb = ((d0* 10**3) / Decimal(7.62))**Decimal(-0.107)
  elif 51 < d0*10**3 <= 254:
    kb = Decimal(1.51) * ((d0* 10**3)**Decimal(-0.157))
  print(f'kb: {kb}')
  lim_normal_res = Decimal(ka)* Decimal(kb) * Decimal(kc) * Decimal(kd) * Decimal(1) * Decimal(kf) * Decimal(0.5) * Decimal(tensao_ruptura)
  lim_cisalhante_res = Decimal(ka)* Decimal(kb) * Decimal(kc) * Decimal(kd) * Decimal(0.577) * Decimal(kf) * Decimal(0.5) * Decimal(tensao_ruptura)

  d1 = newton(f,d0,10e-8,20)

  print(f'd0:{d0} ---- d1:{d1}')
  if abs(d1-d0) < 10e-8:
    parar = True
  else:
    d0 = d1

print(d1)

kb: 0.8018336454984593245684127131
k	 d0		 f(d0)
1	6.890559e-02	2.566982e+08
2	7.792040e-02	5.484899e+07
3	8.106690e-02	4.139514e+06
4	8.134501e-02	2.817344e+04
5	8.134693e-02	1.325935e+00
6	8.134693e-02	0.000000e+00
d0:0.05635192248918816687989031757 ---- d1:0.08134693107770953859709408679
kb: 0.7569259299706397415388034044
k	 d0		 f(d0)
1	8.262766e-02	6.038967e+05
2	8.266938e-02	6.078712e+02
3	8.266942e-02	6.173849e-04
4	8.266942e-02	-5.960464e-08
d0:0.08134693107770953859709408679 ---- d1:0.08266942014948222186497392374
kb: 0.7550119044983874348029175563
k	 d0		 f(d0)
1	8.272821e-02	1.206171e+03
2	8.272830e-02	2.430677e-03
3	8.272830e-02	0.000000e+00
d0:0.08266942014948222186497392374 ---- d1:0.08272829694281911504056828197
kb: 0.7549275178317660371143696251
k	 d0		 f(d0)
1	8.273090e-02	2.354427e+00
2	8.273090e-02	5.960464e-08
d0:0.08272829694281911504056828197 ---- d1:0.08273089769373599455868239686
kb: 0.7549237918417517173888881925
k	 d0		 f(d0)
1	8.273101e-02	4.590929e-03
2	8.27